---
# DataWrangling Comtrade

La siguiente base de datos fue extraida desde UN Comtrade (https://comtrade.un.org/), que correspone a una base de datos con información histórica de comercio entre paises, mantenida por las Naciones Unidas.

Los datos informados correponden a transacciones de Chile y de Perú.

#### 1. Importar librerías

In [2]:
#### 1. Importar librerías
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#### 2. Realice la lectura de datos

In [7]:
#### 2. Realice la lectura de datos
data = pd.read_csv('Comtrade-X-CL-PE.csv', encoding='latin1', sep=',')

#### 3. Realice operaciones básicas de exploración del dataset

In [ ]:
#### 3. Realice operaciones básicas de exploración del dataset
data.head()
data.info()
data.describe()

#### 4. ¿Cuántas filas y columnas tiene el dataset?

In [8]:
#### 4. ¿Cuántas filas y columnas tiene el dataset?
data.shape

(32538, 35)

(32538, 35)

#### 5. ¿Cuántos reporters distintos hay en los datos? (campo Reporter)

In [12]:
#### 5. ¿Cuántos reporters distintos hay en los datos? (campo Reporter)
print(data['Reporter'].nunique())
data['Reporter'].unique()


2


array(['Chile', 'Peru'], dtype=object)

array(['Chile', 'Peru'], dtype=object)

#### 5. ¿Qué años de data considera el reporte?

In [10]:
#### 5. ¿Qué años de data considera el reporte?
data['Year'].unique()

array([2018, 2019])

array([2018, 2019], dtype=int64)

#### 5. Haga un análisis de valores nulos

In [13]:
#### 5. Haga un análisis de valores nulos
data.isnull().sum()

Classification                0
Year                          0
Period                        0
Period Desc.                  0
Aggregate Level               0
Is Leaf Code                  0
Trade Flow Code               0
Trade Flow                    0
Reporter Code                 0
Reporter                      0
Reporter ISO                  0
Partner Code                  0
Partner                       0
Partner ISO                   0
2nd Partner Code              0
2nd Partner                   0
2nd Partner ISO               0
Customs Proc. Code            0
Customs                       0
Mode of Transport Code        0
Mode of Transport             0
Commodity Code                0
Commodity                     0
Qty Unit Code                 0
Qty Unit                  32538
Qty                           0
Alt Qty Unit Code             0
Alt Qty Unit              32538
Alt Qty                       0
Netweight (kg)             4185
Gross weight (kg)          4070
Trade Va

In [ ]:
data.isnull().sum()

Classification                0
Year                          0
Period                        0
Period Desc.                  0
Aggregate Level               0
Is Leaf Code                  0
Trade Flow Code               0
Trade Flow                    0
Reporter Code                 0
Reporter                      0
Reporter ISO                  0
Partner Code                  0
Partner                       0
Partner ISO                   0
2nd Partner Code              0
2nd Partner                   0
2nd Partner ISO               0
Customs Proc. Code            0
Customs                       0
Mode of Transport Code        0
Mode of Transport             0
Commodity Code                0
Commodity                     0
Qty Unit Code                 0
Qty Unit                  32538
Qty                           0
Alt Qty Unit Code             0
Alt Qty Unit              32538
Alt Qty                       0
Netweight (kg)             4185
Gross weight (kg)          4070
Trade Va

#### 6. En base al análisis anterior, ¿eliminaría columnas? ¿cuáles? Si es así, proceda...

In [ ]:
#### 6. En base al análisis anterior, ¿eliminaría columnas? ¿cuáles? Si es así, proceda...
data_cleaned = data.drop(columns=['Netweight (kg)', 'Qty Unit Code', 'Qty Unit', 'CIF (US$)', 'FOB (US$)'])
data_cleaned.info()
'''
Las columnas se eliminan porque:

Netweight (kg), Qty Unit Code, Qty Unit: Tienen muchos valores nulos o no aportan información relevante para el análisis principal.
CIF Trade Value (US$) y FOB Trade Value (US$): Si el análisis se centra en "Trade Value (US$)", estas columnas pueden ser redundantes o tener muchos nulos.
Eliminar columnas con alta cantidad de nulos o irrelevantes ayuda a simplificar el dataset y evitar problemas en el análisis posterior.
'''

#### 7. ¿Que densidad de datos tienen las columnas Netwweight y Trade Value?
Nos referimos al porcentaje de datos no nulo que tiene cada columna respecto al total

In [15]:
#### 7. ¿Que densidad de datos tienen las columnas Netwweight y Trade Value?
#Nos referimos al porcentaje de datos no nulo que tiene cada columna respecto al total
netweight_density = data['Netweight (kg)'].notnull().mean() * 100
trade_value_density = data['Trade Value (US$)'].notnull().mean() * 100
print(f'Densidad de datos en Netweight (kg): {netweight_density:.2f}%')
print(f'Densidad de datos en Trade Value (US$): {trade_value_density:.2f}%')

Densidad de datos en Netweight (kg): 87.14%
Densidad de datos en Trade Value (US$): 100.00%


Netweight (kg)        87.138115
Trade Value (US$)    100.000000
dtype: float64

#### 8. Analice la columna Netweight. ¿Encuentra alguna inconsistencia?

In [ ]:
#### 8. Analice la columna Netweight. ¿Encuentra alguna inconsistencia?
data['Netweight (kg)'].describe()

# Buscar valores negativos o nulos
print("Valores negativos:", (data['Netweight (kg)'] < 0).sum())
print("Valores nulos:", data['Netweight (kg)'].isnull().sum())

'''
Inconsistencias encontradas:
- Hay valores negativos en la columna Netweight (kg), lo cual no tiene sentido físico.
- Existen muchos valores nulos, lo que puede afectar el análisis.
Recomendación: Revisar si los valores negativos corresponden a errores de carga y considerar cómo tratar los nulos.
'''

#### 9. Calcule los límites inferiores y superiores de outlaiers de la columna Trade Value. ¿Qué cantidad posee dicha columna?

In [ ]:
#### 9. Calcule los límites inferiores y superiores de outlaiers de la columna Trade Value. ¿Qué cantidad posee dicha columna?
Q1 = data['Trade Value (US$)'].quantile(0.25)
Q3 = data['Trade Value (US$)'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
outliers = data[(data['Trade Value (US$)'] < lower_bound) | (data['Trade Value (US$)'] > upper_bound)]
print(f'Cantidad de outliers en Trade Value (US$): {outliers.shape[0]}')

IQR: 816586.75
LSUP: 2047498.875
LINF: -1218848.125
Cant Outliers 5720


#### 10. Analice el registro que presenta el máximo valor de Trade Value. ¿A qué corresponde? ¿Podría ser considerado un error o un caso poco frecuente? Fundamente su respuesta

In [17]:
#### 10. Analice el registro que presenta el máximo valor de Trade Value. ¿A qué corresponde? ¿Podría ser considerado un error o un caso poco frecuente? Fundamente su respuesta
max_trade_value_record = data.loc[data['Trade Value (US$)'].idxmax()]
print(max_trade_value_record)

#### 10. Analice el registro que presenta el máximo valor de Trade Value. ¿A qué corresponde? ¿Podría ser considerado un error o un caso poco frecuente? Fundamente su respuesta

max_trade_value_record = data.loc[data['Trade Value (US$)'].idxmax()]
print(max_trade_value_record)

'''
Análisis:
- El registro con el mayor Trade Value corresponde a Chile, partner "World", commodity "Ores, slag and ash" (Código 26).
- Este valor representa el total exportado de ese commodity por Chile en el año, sumando todos los destinos.
- No parece un error, pero sí es un caso poco frecuente porque el partner "World" es un agregado y no un país específico.
- Es importante distinguir estos registros agregados para evitar duplicidades en el análisis.
'''

# # Este comando muestra el registro con el mayor valor de Trade Value, incluyendo el año, país reportante, código y nombre del commodity, partner, modo de transporte y el valor de la transacción.
data[data['Trade Value (US$)']==data['Trade Value (US$)'].max()][['Period','Reporter','Commodity Code','Commodity','Partner','Mode of Transport','Trade Value (US$)']]

Classification                            H5
Year                                    2018
Period                                  2018
Period Desc.                            2018
Aggregate Level                            2
Is Leaf Code                               0
Trade Flow Code                            0
Trade Flow                                 X
Reporter Code                            152
Reporter                               Chile
Reporter ISO                             CHL
Partner Code                               0
Partner                                World
Partner ISO                              W00
2nd Partner Code                           0
2nd Partner                            World
2nd Partner ISO                          W00
Customs Proc. Code                       C00
Customs                             All CPCs
Mode of Transport Code                     0
Mode of Transport                   All MOTs
Commodity Code                            26
Commodity 

,Period,Reporter,Commodity Code,Commodity,Partner,Mode of Transport,Trade Value (US$)
25,2018,Chile,26,"Ores, slag and ash",World,All MOTs,21210039528


In [ ]:
df[df['Trade Value (US$)']==df['Trade Value (US$)'].max()][['Period','Reporter','Commodity Code','Commodity','Partner','Mode of Transport','Trade Value (US$)']]

,Period,Reporter,Commodity Code,Commodity,Partner,Mode of Transport,Trade Value (US$)
25,2018,Chile,26,"Ores, slag and ash",World,All MOTs,21210039528


#### 11. ¿Cuáles son los diferentes partners que Chile realiza sus exportaciones? ¿Qué le llama la atención del resultado?

In [21]:
#### 11. ¿Cuáles son los diferentes partners que Chile realiza sus exportaciones? ¿Qué le llama la atención del resultado?
print(data['Trade Flow'].unique())
print(data['Reporter'].unique())


chile_partners = data[(data['Reporter'] == 'Chile') & (data['Trade Flow'] == 'Export')]['Partner'].unique()
print(chile_partners)

chile_partners = data[(data['Reporter'] == 'Chile') & (data['Trade Flow'] == 'X')]['Partner'].unique()
print(chile_partners)

['X']
['Chile' 'Peru']
[]
['World' 'Algeria' 'Andorra' 'Angola' 'Antigua and Barbuda' 'Azerbaijan'
 'Argentina' 'Australia' 'Austria' 'Bahamas' 'Bahrain' 'Bangladesh'
 'Armenia' 'Barbados' 'Belgium' 'Bermuda'
 'Bolivia (Plurinational State of)' 'Bosnia Herzegovina' 'Botswana'
 'Brazil' 'Belize' 'Br. Virgin Isds' 'Brunei Darussalam' 'Bulgaria'
 'Myanmar' 'Belarus' 'Cambodia' 'Cameroon' 'Canada' 'Cabo Verde'
 'Cayman Isds' 'Sri Lanka' 'China' 'Colombia' 'Comoros' 'Congo'
 'Dem. Rep. of the Congo' 'Cook Isds' 'Costa Rica' 'Croatia' 'Cuba'
 'Cyprus' 'Czechia' 'Benin' 'Denmark' 'Dominica' 'Dominican Rep.'
 'Ecuador' 'El Salvador' 'Equatorial Guinea' 'Ethiopia' 'Estonia' 'Fiji'
 'Finland' 'France' 'French Polynesia' 'Fr. South Antarctic Terr.'
 'Djibouti' 'Gabon' 'Georgia' 'Germany' 'Ghana' 'Kiribati' 'Greece'
 'Grenada' 'Guam' 'Guatemala' 'Guinea' 'Guyana' 'Haiti' 'Honduras'
 'China, Hong Kong SAR' 'Hungary' 'Iceland' 'Indonesia' 'Iran' 'Iraq'
 'Ireland' 'Israel' 'Italy' "CÃ´te d'Ivoire" 'J

array(['World', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda',
       'Azerbaijan', 'Argentina', 'Australia', 'Austria', 'Bahamas',
       'Bahrain', 'Bangladesh', 'Armenia', 'Barbados', 'Belgium',
       'Bermuda', 'Bolivia (Plurinational State of)',
       'Bosnia Herzegovina', 'Botswana', 'Brazil', 'Belize',
       'Br. Virgin Isds', 'Brunei Darussalam', 'Bulgaria', 'Myanmar',
       'Belarus', 'Cambodia', 'Cameroon', 'Canada', 'Cabo Verde',
       'Cayman Isds', 'Sri Lanka', 'China', 'Colombia', 'Comoros',
       'Congo', 'Dem. Rep. of the Congo', 'Cook Isds', 'Costa Rica',
       'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Benin', 'Denmark',
       'Dominica', 'Dominican Rep.', 'Ecuador', 'El Salvador',
       'Equatorial Guinea', 'Ethiopia', 'Estonia', 'Fiji', 'Finland',
       'France', 'French Polynesia', 'Fr. South Antarctic Terr.',
       'Djibouti', 'Gabon', 'Georgia', 'Germany', 'Ghana', 'Kiribati',
       'Greece', 'Grenada', 'Guam', 'Guatemala', 'Guinea', 'Guyana',
 

#### Análisis del partner "World"

Como se pudo apreciar, el partner "World" es un valor que requiere mayor atención. Vamos a crear una nueva columna en el dataframe donde señalaremos si correponde a un registro "NotWorld:

In [ ]:
data['NotWorld'] = data['Partner'].apply(lambda x : x !='World')

Ahora vamos a sumarizar los valores de registros World y NotWorld, lo haremos tomando como referencia el código de commodity 26

In [ ]:
data[data['Commodity Code']==26].groupby('NotWorld').sum()[['Trade Value (US$)','FOB Trade Value (US$)']]

,Trade Value (US$),FOB Trade Value (US$)
NotWorld,,
False,111495065061,1.114951e+11
True,111495064956,1.114951e+11


Como se puede apreciar, el registro World totaliza los registros de los demás países. En la siguiente tabla pivoteada se aprecia con mayor claridad.

In [ ]:
data[data['Commodity Code']==26].pivot_table(index='Partner', columns='NotWorld', 
                                         values='Trade Value (US$)', aggfunc='sum', 
                                                                        margins=True)

NotWorld,False,True,All
Partner,,,
Algeria,NaN,3.388939e+07,33889390
Argentina,NaN,2.498466e+07,24984664
Aruba,NaN,1.206900e+04,12069
Australia,NaN,3.177558e+08,317755802
Austria,NaN,1.410000e+02,141
Bahrain,NaN,7.203845e+07,72038449
Belgium,NaN,9.718908e+08,971890842
Bolivia (Plurinational State of),NaN,1.417090e+05,141709
Brazil,NaN,3.029797e+09,3029797286


#### 12. Elimine los registros con partner World

In [ ]:
#### 12. Elimine los registros con partner World
data_no_world = data[data['Partner'] != 'World']

#### 13. Vuelva a desplegar el registro con el mayor valor de Trade Value. ¿Qué diferencia se aprecia? ¿A quién le realiza la mayor exportación de ese commodity?

In [ ]:
#### 13. Vuelva a desplegar el registro con el mayor valor de Trade Value. ¿Qué diferencia se aprecia? ¿A quién le realiza la mayor exportación de ese commodity?
data_no_world[data_no_world['Trade Value (US$)']==data_no_world['Trade Value (US$)'].max()][['Period','Reporter','Commodity Code','Commodity','Partner','Mode of Transport','Trade Value (US$)']]

,Period,Reporter,Commodity Code,Commodity,Partner,Mode of Transport,Trade Value (US$)
743,2018,Chile,26,"Ores, slag and ash",China,All MOTs,10678481900


#### Análisis columna Mode of Transport
Ahora encontramos que la columna Mode of Transport tiene el valor All MOTs, es posible que suceda lo mismo que con partner World. Antes de eliminar la columna, verificaremos si tanto Chile como Perú reportan sus exportaciones indicando el medio de transporte.

In [ ]:
data['NotAllMOTs'] = data['Partner'].apply(lambda x : x !='All MOTs')

# tomamos como ejemplo los commodities de Animales y carne (commodity code = 2)
data[data['Commodity Code']==2].pivot_table(index=['Reporter','Mode of Transport'], 
                                        columns='NotAllMOTs', 
                                        values='Trade Value (US$)', 
                                        aggfunc='sum', margins=True)


NotAllMOTs                        True         All
Reporter Mode of Transport                        
Chile    All MOTs           2101889437  2101889437
Peru     Air                    193991      193991
         All MOTs              9595133     9595133
         Sea                   9401142     9401142
All                         2121079703  2121079703

En este caso, se aprecia que Chile no reporta sus exportaciones desglosando el medio de transporte pero Perú sí lo hace. Si hubiéramos eliminado todos los registros de All MOTs, habríamos borrado los datos de Chile...

#### 15. Elimine los registros All MOTs solamente para el caso de Perú

#### 16. Vuelva a desplegar el registro que presenta el máximo valor de Trade Value. ¿Quién es el principal partner de exportación de Chile, qué commodity se exporta y cuál es el medio de transporte?

,Period,Reporter,Commodity Code,Commodity,Partner,Mode of Transport,Trade Value (US$)
743,2018,Chile,26,"Ores, slag and ash",China,All MOTs,10678481900


#### 17. ¿Quién es el principal partner de exportaciones de Perú, qué commodity es y cuál es el medio de transporte?

,Period,Reporter,Commodity Code,Commodity,Partner,Mode of Transport,Trade Value (US$)
23004,2019,Peru,26,"Ores, slag and ash",China,Sea,10618326550


#### 18. Cree una columna adicional (Is Leather), booleana, con valor True si el commodity importado está relacionado con cuero (Leather). Posteriormente, contabilice cuántos registros tienen esta marca en True

395

#### 19. ¿Cuánto es el valor de las exportaciones de cuero de Chile el año 2019?

48934992

## Buen trabajo!!

---